In [ ]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
#!/usr/bin/env python
# coding: utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
import glob
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from tqdm import tqdm_notebook, tqdm
import json
from typing import NamedTuple
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
print(torch.__version__)
# from tools import eval_summary, save_feature_importance, merge_preds

device = torch.device('cpu')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
torch.set_num_threads(16)

In [ ]:
torch.get_num_threads()

#### Load Data

In [ ]:
df_train = pd.read_csv('input/train.csv', dtype=np.float32)
df_test = pd.read_csv('input/test.csv', dtype=np.float32)
print(df_train.shape, df_test.shape)


In [ ]:
layer_cols = [c for c in df_train.columns if 'layer_' in c]
fea_cols = [c for c in df_train.columns if c not in layer_cols]

len(fea_cols), len(layer_cols)

In [ ]:
df_model = df_train

#### Model

In [ ]:
class DNN1Model(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super().__init__()
        relu = torch.nn.ReLU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.layer_1 = torch.nn.Sequential(
            torch.nn.Linear(input_size + 1, input_size), relu, 
            torch.nn.Linear(input_size, input_size), relu, 
            torch.nn.Linear(input_size, input_size), relu, 
        )
        self.layer_2 = torch.nn.Sequential(
            torch.nn.Linear(input_size + 1, input_size), relu, 
            torch.nn.Linear(input_size, input_size), relu, 
            torch.nn.Linear(input_size, input_size), relu, 
        )
        self.layer_3 = torch.nn.Sequential(
            torch.nn.Linear(input_size + 1, input_size), relu, 
            torch.nn.Linear(input_size, input_size), relu, 
            torch.nn.Linear(input_size, input_size), relu, 
        )
        self.layer_4 = torch.nn.Sequential(
            torch.nn.Linear(input_size + 1, input_size), relu, 
            torch.nn.Linear(input_size, input_size), relu, 
            torch.nn.Linear(input_size, input_size),
        )
    def forward(self, x_fea, x1, x2, x3, x4):
        out_layer_1 = self.layer_1(torch.cat([x_fea, x1], axis=1))
        out_layer_2 = self.layer_2(torch.cat([out_layer_1, x2], axis=1))
        out_layer_3 = self.layer_3(torch.cat([out_layer_2, x3], axis=1))
        out_layer_4 = self.layer_4(torch.cat([out_layer_3, x4], axis=1))
        
        return out_layer_4
    

#### Train

In [ ]:
# model.load_state_dict(torch.load('checkpoint.pt'))

In [ ]:
class SemiDataset(Dataset):
    def __init__(self, df, fea_cols, layer_cols):        
        self.fea_cols = fea_cols
        self.X_fea = df[fea_cols].values
        self.X_layers = []
        for c in layer_cols:
            self.X_layers.append(df_model[[c]].values)
        self.y = np.zeros((len(self.X_fea), len(self.fea_cols)))

    def __len__(self):
        return len(self.X_fea)
    
    def __getitem__(self, idx):
        layers = [x[idx] for x in self.X_layers]
#         return self.X_fea[idx].astype(np.float32), layers, self.y[idx].astype(np.float32)
        return self.y[idx].astype(np.float32), layers, self.X_fea[idx].astype(np.float32)


In [ ]:
model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
print(model_ts)

print(f'fea_size {len(fea_cols)} layer_cols {layer_cols}')

model = DNN1Model(input_size=len(fea_cols), dropout_probability=0.5).to(device)
    
criterion = nn.L1Loss(reduction='mean').to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
scheduler = StepLR(optimizer, step_size=400, gamma=1.0)


In [ ]:
train_dataset = SemiDataset(df_model, fea_cols, layer_cols)

train_loader_params = {
    'dataset' : train_dataset,
    'batch_size' : len(train_dataset) // 4,
    'shuffle' : True,
    'num_workers' : 4,
    'drop_last' : False,
}
train_loader = DataLoader(**train_loader_params)


In [ ]:
total_epoch = 100
model.train()

for e in tqdm_notebook(range(total_epoch), total=total_epoch, desc='Epoch'):
    total_loss = 0
    for data in train_loader:
        X_fea, X_layers, y_batch = data
        
        y_pred = model(X_fea.to(device), *[x.to(device) for x in X_layers])

        loss = criterion(y_pred, y_batch.to(device))
        total_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #     scheduler.step()
    
    ts = datetime.now().strftime('%Y%m%dT%H%M%S')
    print(f'[{ts}] Epock {e} / {total_epoch} loss: {total_loss / len(train_loader)}')
    
    if e % 1000 == 0:
        torch.save(model.state_dict(), 'checkpoint_1nn.pt')

torch.save(model.state_dict(), 'checkpoint_1nn.pt')

In [ ]:
# self.X_fea = df[fea_cols].values
# self.X_layers = []
# for c in layer_cols:
#     self.X_layers.append(df_model[[c]].values)

In [ ]:
X_test = torch.Tensor(df_model.loc[:1, fea_cols].values)
X_test_layers = []
for c in layer_cols:
    X_test_layers.append(torch.Tensor(df_model.loc[:1, [c]].values))
X_test_layers



In [ ]:
y_test = torch.Tensor(np.zeros(len(fea_cols)).astype(np.float32))

In [ ]:
y_test

In [ ]:
y_pred = model(X_test.to(device), *[x.to(device) for x in X_test_layers])

In [ ]:
criterion(y_pred[0], y_test.to(device))

In [ ]:
criterion(y_pred[1], y_test.to(device))